In [11]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df_24= pd.read_csv("flamengo_2024.csv")
df_25 = pd.read_csv("flamengo_2025.csv")
df_24.head()


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-04-02,17:00,Libertadores,Group stage,Tue,Away,D,1,1,co Millonarios,NaN,NaN,54,22205.0,Giorgian De Arrascaeta,4-2-3-1,4-4-2,NaN,Match Report,NaN
1,2024-04-10,21:30,Libertadores,Group stage,Wed,Home,W,2,0,cl Palestino,NaN,NaN,63,NaN,Giorgian De Arrascaeta,4-2-3-1,4-2-3-1,NaN,Match Report,NaN
2,2024-04-14,16:00,Série A,Matchweek 1,Sun,Away,W,2,1,Atl Goianiense,1.2,1.6,67,31617.0,Giorgian De Arrascaeta,4-2-3-1,4-1-3-2,NaN,Match Report,NaN
3,2024-04-17,21:30,Série A,Matchweek 2,Wed,Home,W,2,1,São Paulo,1.3,0.7,47,45166.0,Bruno Henrique,4-2-3-1,3-4-3,NaN,Match Report,NaN
4,2024-04-21,16:00,Série A,Matchweek 3,Sun,Away,D,0,0,Palmeiras,0.2,0.4,50,29956.0,Giorgian De Arrascaeta,4-2-3-1,3-4-3,NaN,Match Report,NaN


In [4]:
# Convertendo 'Attendance' para valor numérico
df_24['Attendance'] = pd.to_numeric(df_24['Attendance'], errors='coerce')

# Substituindo valores NaN em xG e xGA pela média arredondada a 2 casas decimais
df_24[['xG', 'xGA']] = df_24[['xG', 'xGA']].apply(lambda col: col.fillna(np.round(col.mean(), 2)))

# Calculando a média de Attendance ignorando NaN
attendance_mean = df_24['Attendance'].mean()

# Substituindo NaN em Attendance pela média ded publico nos estádios
df_24['Attendance'] = df_24['Attendance'].fillna(np.round(attendance_mean, 0) if not np.isnan(attendance_mean) else 0)

# as colunas "notes" e "referee" estão todas em NaN, mas não são importantes para a nossa analise por isso vamos exclui-la
df_24.drop(columns=['Referee', 'Notes'], inplace=True)


In [6]:
# Convertendo a coluna 'Date' para o formato datetime
df_24['Date'] = pd.to_datetime(df_24['Date'], format='%Y-%m-%d')

data_corte = "2024-09-30"  # Formato correto
data_corte = pd.to_datetime(data_corte)

# Criando dois DataFrames separados
df_tite = df_24[df_24['Date'] <= data_corte]
df_felipe_luis = df_24[df_24['Date'] > data_corte]

# Unindo os dados
df_2025 = pd.concat([df_felipe_luis, df_25], ignore_index=True)

# Mostrando os dados do período do Tite
print("Período do Tite: ", df_tite)

df_tite.head()


Período do Tite:           Date   Time          Comp           Round  Day Venue Result  GF  GA  \
0  2024-04-02  17:00  Libertadores     Group stage  Tue  Away      D   1   1   
1  2024-04-10  21:30  Libertadores     Group stage  Wed  Home      W   2   0   
2  2024-04-14  16:00       Série A     Matchweek 1  Sun  Away      W   2   1   
3  2024-04-17  21:30       Série A     Matchweek 2  Wed  Home      W   2   1   
4  2024-04-21  16:00       Série A     Matchweek 3  Sun  Away      D   0   0   
5  2024-04-24  20:30  Libertadores     Group stage  Wed  Away      L   1   2   
6  2024-04-28  11:00       Série A     Matchweek 4  Sun  Home      L   0   2   
7  2024-05-04  18:30       Série A     Matchweek 5  Sat  Away      D   1   1   
8  2024-05-07  20:00  Libertadores     Group stage  Tue  Away      L   0   1   
9  2024-05-11  16:00       Série A     Matchweek 6  Sat  Home      W   2   0   
10 2024-05-15  21:30  Libertadores     Group stage  Wed  Home      W   4   0   
11 2024-05-28  21:00  

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Match Report
0,2024-04-02,17:00,Libertadores,Group stage,Tue,Away,D,1,1,co Millonarios,1.48,1.06,54,22205.0,Giorgian De Arrascaeta,4-2-3-1,4-4-2,Match Report
1,2024-04-10,21:30,Libertadores,Group stage,Wed,Home,W,2,0,cl Palestino,1.48,1.06,63,44292.0,Giorgian De Arrascaeta,4-2-3-1,4-2-3-1,Match Report
2,2024-04-14,16:00,Série A,Matchweek 1,Sun,Away,W,2,1,Atl Goianiense,1.20,1.60,67,31617.0,Giorgian De Arrascaeta,4-2-3-1,4-1-3-2,Match Report
3,2024-04-17,21:30,Série A,Matchweek 2,Wed,Home,W,2,1,São Paulo,1.30,0.70,47,45166.0,Bruno Henrique,4-2-3-1,3-4-3,Match Report
4,2024-04-21,16:00,Série A,Matchweek 3,Sun,Away,D,0,0,Palmeiras,0.20,0.40,50,29956.0,Giorgian De Arrascaeta,4-2-3-1,3-4-3,Match Report


In [124]:
# começando analises do flamengo de Tite

print(" ======================= Analises do flamengo de Tite =======================")
# Resultados com o tite
df_derrotas = df_tite[df_tite["Result"] == "L"].shape[0]
df_empates = df_tite[df_tite["Result"] == "D"].shape[0]
df_vitorias = df_tite[df_tite["Result"] == "W"].shape[0]

print(f"O flamengo de tite deve: ", df_derrotas, " derrotas, ", df_empates, " empates e ", df_vitorias , " vitorias")

# como era o desempenho tático?
# média de posse de bola
media_posse = df_tite["Poss"].mean()
print(f"Média de posse de bola do Flamengo de Tite: {media_posse:.2f}")

# formação mais usada
formacao_mais_usada = df_tite["Formation"].max()
# formação menos usada
formacao_menos_usada = df_tite["Formation"].min()
print(f"Formação mais usada: ", formacao_mais_usada, "VS formacao menos usada: ", formacao_menos_usada )

# Média de xG, gols esperados
media_xG = df_tite["xG"].mean()
print(f"Média de xG:  {media_xG:.2f}")

# Média de xAG, gols esperados que aconteceram
media_xGA = df_tite["xGA"].mean()
print(f"Média xGA: {media_xGA:.2f}")




# Desempenho em casa
print("\n === Desempenho em casa === ")

# Número total de jogos em casa
total_jogos_casa = df_tite[df_tite["Venue"] == "Home"].shape[0]

# Número de vitórias em casa
vitorias_casa = df_tite[(df_tite["Venue"] == "Home") & (df_tite["Result"] == "W")].shape[0]

# Média de vitórias em casa (percentual de vitórias)
percentual_vitorias_casa = (vitorias_casa / total_jogos_casa) * 100 if total_jogos_casa > 0 else 0
print(f"\nPorcentagem de vitórias em casa: {percentual_vitorias_casa:.2f}%")

# Média de xG em casa
media_xG_casa = df_tite[df_tite["Venue"] == "Home"]["xG"].mean()
print(f"Média de xG em casa: {media_xG_casa:.2f}")

# Gols feitos em casa
gols_feitos_casa = df_tite[df_tite["Venue"] == "Home"]["GF"].sum()
gols_feitos_casa_media = gols_feitos_casa / total_jogos_casa if total_jogos_casa > 0 else 0
print(f"Gols feitos em casa (Total): {gols_feitos_casa}")
print(f"Gols feitos em casa (Média): {gols_feitos_casa_media:.2f}")

# Gols concedidos em casa
gols_concedidos_casa = df_tite[df_tite["Venue"] == "Home"]["GA"].sum()
gols_concedidos_casa_media = gols_concedidos_casa / total_jogos_casa if total_jogos_casa > 0 else 0
print(f"Gols concedidos em casa (Total): {gols_concedidos_casa}")
print(f"Gols concedidos em casa (Média): {gols_concedidos_casa_media:.2f}")


 ======================= Analises do flamengo de Tite =======================
O flamengo de tite deve:  11  derrotas,  8  empates e  18  vitorias
Média de posse de bola do Flamengo de Tite: 55.14
Formação mais usada:  4-4-2 VS formacao menos usada:  3-4-3
Média de xG:  1.39
Média xGA: 1.12

 === Desempenho em casa === 

Porcentagem de vitórias em casa: 68.42%
Média de xG em casa: 1.37
Gols feitos em casa (Total): 32
Gols feitos em casa (Média): 1.68
Gols concedidos em casa (Total): 14
Gols concedidos em casa (Média): 0.74


In [16]:
# começando análises do Flamengo de Tite (fl)

print(" ======================= Análises do Flamengo de Tite (fl) =======================")
# Resultados com o Tite (fl)
df_derrotas_fl = df_tite[df_tite["Result"] == "L"].shape[0]
df_empates_fl = df_tite[df_tite["Result"] == "D"].shape[0]
df_vitorias_fl = df_tite[df_tite["Result"] == "W"].shape[0]

print(f"O Flamengo de Tite (fl) teve: ", df_derrotas_fl, " derrotas, ", df_empates_fl, " empates e ", df_vitorias_fl , " vitórias")

# Como era o desempenho tático?
# Média de posse de bola
media_posse_fl = df_tite["Poss"].mean()
print(f"Média de posse de bola do Flamengo de Tite (fl): {media_posse_fl:.2f}")

# Formação mais usada
formacao_mais_usada_fl = df_tite["Formation"].max()
# Formação menos usada
formacao_menos_usada_fl = df_tite["Formation"].min()
print(f"Formação mais usada: ", formacao_mais_usada_fl, "VS formação menos usada: ", formacao_menos_usada_fl)

# Média de xG, gols esperados
media_xG_fl = df_tite["xG"].mean()
print(f"Média de xG:  {media_xG_fl:.2f}")

# Média de xGA, gols esperados que aconteceram
media_xGA_fl = df_tite["xGA"].mean()
print(f"Média xGA: {media_xGA_fl:.2f}")

# Desempenho em casa
print("\n === Desempenho em casa === ")

# Número total de jogos em casa
total_jogos_casa_fl = df_tite[df_tite["Venue"] == "Home"].shape[0]

# Número de vitórias em casa
vitorias_casa_fl = df_tite[(df_tite["Venue"] == "Home") & (df_tite["Result"] == "W")].shape[0]

# Média de vitórias em casa (percentual de vitórias)
percentual_vitorias_casa_fl = (vitorias_casa_fl / total_jogos_casa_fl) * 100 if total_jogos_casa_fl > 0 else 0
print(f"\nPorcentagem de vitórias em casa: {percentual_vitorias_casa_fl:.2f}%")

# Média de xG em casa
media_xG_casa_fl = df_tite[df_tite["Venue"] == "Home"]["xG"].mean()
print(f"Média de xG em casa: {media_xG_casa_fl:.2f}")

# Gols feitos em casa
gols_feitos_casa_fl = df_tite[df_tite["Venue"] == "Home"]["GF"].sum()
gols_feitos_casa_media_fl = gols_feitos_casa_fl / total_jogos_casa_fl if total_jogos_casa_fl > 0 else 0
print(f"Gols feitos em casa (Total): {gols_feitos_casa_fl}")
print(f"Gols feitos em casa (Média): {gols_feitos_casa_media_fl:.2f}")

# Gols concedidos em casa
gols_concedidos_casa_fl = df_tite[df_tite["Venue"] == "Home"]["GA"].sum()
gols_concedidos_casa_media_fl = gols_concedidos_casa_fl / total_jogos_casa_fl if total_jogos_casa_fl > 0 else 0
print(f"Gols concedidos em casa (Total): {gols_concedidos_casa_fl}")
print(f"Gols concedidos em casa (Média): {gols_concedidos_casa_media_fl:.2f}")

tite_jogos = df_tite["Comp"].count() # Contando o número de jogos usando uma coluna qualquer como medida
tite_jogos_casa = df_tite[df_tite["Venue"] == "Home"].shape[0] # Contando o número de jogos em casa



 ======================= Análises do Flamengo de Tite (fl) =======================
O Flamengo de Tite (fl) teve:  11  derrotas,  8  empates e  18  vitórias
Média de posse de bola do Flamengo de Tite (fl): 55.14
Formação mais usada:  4-4-2 VS formação menos usada:  3-4-3
Média de xG:  1.39
Média xGA: 1.12

 === Desempenho em casa === 

Porcentagem de vitórias em casa: 68.42%
Média de xG em casa: 1.37
Gols feitos em casa (Total): 32
Gols feitos em casa (Média): 1.68
Gols concedidos em casa (Total): 14
Gols concedidos em casa (Média): 0.74


In [14]:
# começando analises do flamengo de Felipe Luís (fl)

print(" ======================= Analises do flamengo de Felipe Luís (fl) =======================")
# Resultados com o Felipe Luís (fl)
df_derrotas_fl = df_felipe_luis[df_felipe_luis["Result"] == "L"].shape[0]
df_empates_fl = df_felipe_luis[df_felipe_luis["Result"] == "D"].shape[0]
df_vitorias_fl = df_felipe_luis[df_felipe_luis["Result"] == "W"].shape[0]

print(f"O flamengo de Felipe Luís (fl) deve: ", df_derrotas_fl, " derrotas, ", df_empates_fl, " empates e ", df_vitorias_fl , " vitorias")

# como era o desempenho tático?
# média de posse de bola
media_posse_fl = df_felipe_luis["Poss"].mean()
print(f"Média de posse de bola do Flamengo de Felipe Luís (fl): {media_posse_fl:.2f}")

# formação mais usada
formacao_mais_usada_fl = df_felipe_luis["Formation"].max()
# formação menos usada
formacao_menos_usada_fl = df_felipe_luis["Formation"].min()
print(f"Formação mais usada: ", formacao_mais_usada_fl, "VS formação menos usada: ", formacao_menos_usada_fl)

# Média de xG, gols esperados
media_xG_fl = df_felipe_luis["xG"].mean()
print(f"Média de xG:  {media_xG_fl:.2f}")

# Média de xGA, gols esperados que aconteceram
media_xGA_fl = df_felipe_luis["xGA"].mean()
print(f"Média xGA: {media_xGA_fl:.2f}")


# Desempenho em casa
print("\n === Desempenho em casa === ")

# Número total de jogos em casa
total_jogos_casa_fl = df_felipe_luis[df_felipe_luis["Venue"] == "Home"].shape[0]

# Número de vitórias em casa
vitorias_casa_fl = df_felipe_luis[(df_felipe_luis["Venue"] == "Home") & (df_felipe_luis["Result"] == "W")].shape[0]

# Média de vitórias em casa (percentual de vitórias)
percentual_vitorias_casa_fl = (vitorias_casa_fl / total_jogos_casa_fl) * 100 if total_jogos_casa_fl > 0 else 0
print(f"\nPorcentagem de vitórias em casa: {percentual_vitorias_casa_fl:.2f}%")

# Média de xG em casa
media_xG_casa_fl = df_felipe_luis[df_felipe_luis["Venue"] == "Home"]["xG"].mean()
print(f"Média de xG em casa: {media_xG_casa_fl:.2f}")

# Gols feitos em casa
gols_feitos_casa_fl = df_felipe_luis[df_felipe_luis["Venue"] == "Home"]["GF"].sum()
gols_feitos_casa_media_fl = gols_feitos_casa_fl / total_jogos_casa_fl if total_jogos_casa_fl > 0 else 0
print(f"Gols feitos em casa (Total): {gols_feitos_casa_fl}")
print(f"Gols feitos em casa (Média): {gols_feitos_casa_media_fl:.2f}")

# Gols concedidos em casa
gols_concedidos_casa_fl = df_felipe_luis[df_felipe_luis["Venue"] == "Home"]["GA"].sum()
gols_concedidos_casa_media_fl = gols_concedidos_casa_fl / total_jogos_casa_fl if total_jogos_casa_fl > 0 else 0
print(f"Gols concedidos em casa (Total): {gols_concedidos_casa_fl}")
print(f"Gols concedidos em casa (Média): {gols_concedidos_casa_media_fl:.2f}")
felipe_luis_jogos = df_felipe_luis["Comp"].count() # contando o número de jogos usando uma coluna qualquer como medida
felipe_luis_jogos_casa = df_felipe_luis[df_felipe_luis["Venue"] == "Home"].shape[0] # Contando o número de jogos em casa


print("\nO Felipe luis em menos jogos teve uma meia e gols esperados maior, mas menor taxa de conversão")
print("A porcentagem de vitórias em casa é baixa ")
print(f"Mas devemos levar em consideração que são dados do final do campeonato de 2024 e o primeiro jogo de 2025, totalizando", felipe_luis_jogos, " Jogos no total")
print("Alto média de gols em casa, são 9 em ", felipe_luis_jogos_casa,"jogos, quase 2 por jogo")
print("Média de gols em casa maior, temos um time mais efetivo e mais ofensivo também")
print("Até agora a média de gols consedidos é maior, mas em menos numero de jogos e gols no total (8) ")
print("por enquanto mostrando ser um time melhor ofensivamente e defensivamente")


 ======================= Analises do flamengo de Felipe Luís (fl) =======================
O flamengo de Felipe Luís (fl) deve:  1  derrotas,  4  empates e  6  vitorias
Média de posse de bola do Flamengo de Felipe Luís (fl): 59.00
Formação mais usada:  4-4-2 VS formação menos usada:  3-4-3
Média de xG:  1.78
Média xGA: 0.85

 === Desempenho em casa === 

Porcentagem de vitórias em casa: 40.00%
Média de xG em casa: 2.00
Gols feitos em casa (Total): 9
Gols feitos em casa (Média): 1.80
Gols concedidos em casa (Total): 8
Gols concedidos em casa (Média): 1.60

O Felipe luis em menos jogos teve uma meia e gols esperados maior, mas menor taxa de conversão
A porcentagem de vitórias em casa é baixa 
Mas devemos levar em consideração que são dados do final do campeonato de 2024 e o primeiro jogo de 2025, totalizando 11  Jogos no total
Alto média de gols em casa, são 9 em  5 jogos, quase 2 por jogo
Média de gols em casa maior, temos um time mais efetivo e mais ofensivo também
Até agora a média de 

In [22]:
# iniciando estudos com Machine Learning
# o modelo está com apenas 11 jogos para disponibilizar como dados, então o modelo deve ser pouco preciso
# por isso, será apenas uma estimativa

print(df_felipe_luis.head())
print(df_felipe_luis.info())
print(print(df_felipe_luis["Result"].unique()))

# criando variavel para analisar separadamente
felipe_ml = df_felipe_luis.copy()

# mapeando os resultados para numeros
resultado_map = {'W': 2, 'D':1, 'L':0}
felipe_ml['Result'] = felipe_ml['Result'].map(resultado_map)

# removendo colunas que não vão ser usadas
colunas_removidas = ['Date', 'Time', 'Comp', 'Round', 'Day', 'Match Report', 'Captain']
felipe_ml.drop(columns=colunas_removidas, inplace=True)

# transformando outras colunas em numeros
# Verificar quais colunas categóricas ainda existem no DataFrame
coluna_para_numero = ['Venue', 'Result', 'Opponent', 'Formation', 'Opp Formation']

# Filtrar apenas as colunas que ainda existem
coluna_para_numero = [col for col in coluna_para_numero if col in felipe_ml.columns]

print("Colunas a serem transformadas:", coluna_para_numero)

# Aplicar LabelEncoder apenas nas colunas que existem
label_encoders = {}
for col in coluna_para_numero:
    le = LabelEncoder()
    felipe_ml[col] = le.fit_transform(felipe_ml[col])
    label_encoders[col] = le


# separando features e target
X = felipe_ml.drop(columns=['Result'])
Y = felipe_ml['Result']

# treino e teste(80% treino e 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# criando modelo ml
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# previsões para avaliar o modelo
y_pred = modelo.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("A precisão é: ", accuracy)
# a precisão não é satisfatória, por motivos de falta de dados disponiveis, então não vamos conseguir fazer uma analise convincente





         Date   Time     Comp         Round  Day Venue Result  GF  GA  \
37 2024-10-05  19:00  Série A  Matchweek 29  Sat  Away      W   2   0   
38 2024-10-17  20:00  Série A  Matchweek 30  Thu  Home      L   0   2   
39 2024-10-26  16:30  Série A  Matchweek 31  Sat  Home      W   4   2   
40 2024-10-30  19:00  Série A  Matchweek 17  Wed  Away      D   1   1   
41 2024-11-06  21:00  Série A  Matchweek 32  Wed  Away      W   1   0   

         Opponent   xG  xGA  Poss  Attendance         Captain Formation  \
37          Bahia  2.5  0.8    53     43433.0          Gerson   4-2-3-1   
38     Fluminense  1.5  1.7    60     58117.0  Bruno Henrique   4-2-3-1   
39      Juventude  2.4  0.6    74     65700.0          Gerson   4-2-3-1   
40  Internacional  1.7  1.5    53     43729.0          Gerson   4-2-3-1   
41       Cruzeiro  0.4  0.7    48     20855.0  Bruno Henrique   4-2-3-1   

   Opp Formation  Match Report  
37       4-1-3-2  Match Report  
38       4-2-3-1  Match Report  
39       4-